In [ ]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 7.5 MB/s 


In [ ]:
import torch
from google.colab.patches import cv2_imshow
import cv2
import pandas as pd

temp = []
model = torch.hub.load('ultralytics/yolov5', 'yolov5n')

cap = cv2.VideoCapture('inputVideo.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('outputVideo.avi',cv2.VideoWriter_fourcc('M','J','P','G'), cap.get(5), (frame_width,frame_height))
# Minor issue: original video 29 secs but the output video is 41 secs. The FPS of it affects the length.
# I used cap.get(5) which is the video's FPS. If change to a higher fps e.g. 60, the video became shorter (20sec) 
while True:
    img = cap.read()[1]
    cap.set(cv2.CAP_PROP_FPS, 60)
    #fps = int(cap.get(5))
    #print("fps:", fps)
    if img is None:
        break
    result = model(img)
    df = result.pandas().xyxy[0]
    temp.append(df)

    for ind in df.index:
        x1, y1 = int(df['xmin'][ind]), int(df['ymin'][ind])
        x2, y2 = int(df['xmax'][ind]), int(df['ymax'][ind])
        label = df['name'][ind]
        conf = df['confidence'][ind]
        text = label + ' ' + str(conf.round(decimals= 2))
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 0), 2)
        cv2.putText(img, text, (x1, y1 - 5), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
        out.write(img)

df_res = pd.concat(temp)
df_res.to_csv('outputVideo.csv')
#df.to_csv('out.csv')     
#print(df)
    # cv2_imshow(img)
    # cv2.waitKey(10)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-17 Python-3.7.14 torch-1.12.1+cu113 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 
